In [1]:
%pylab inline
from mendeleev import element
from mendeleev.fetch import fetch_table
import pandas as pd
import time

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [2]:
site_map={'A':['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
                 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                 'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb'],
           'B':['Ge','Si','P'],
           'X':['S','Se','Te']}

import re

def get_elements(formula):
    return re.split(r"\d+",formula)[:-1]

formulas=df["formulas"]

mat_els=[get_elements(f) for f in formulas]

A1a=[]
A2a=[]
B1a=[]
B2a=[]
Xa=[]

for els in mat_els:
    As=[el for el in els if el in site_map["A"]]
    Bs=[el for el in els if el in site_map["B"]]
    Xs=[el for el in els if el in site_map["X"]]
    if len(As)==1:
        As.append(As[0])
    if len(Bs)==1:
        Bs.append(Bs[0])
    full_els=As+Bs+Xs
    A1a.append(full_els[0])
    A2a.append(full_els[1])
    B1a.append(full_els[2])
    B2a.append(full_els[3])
    Xa.append(full_els[4])

df["A1"]=A1a
df["A2"]=A2a
df["B1"]=B1a
df["B2"]=B2a
df["X"]=Xa

NameError: name 'df' is not defined

In [ ]:
from pymatgen.core.composition import Composition
import itertools

site_map={'A':['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
                 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                 'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb'],
           'B':['Ge','Si','P'],
           'X':['S','Se','Te']}

mat_els=[['Nb', 'Mo', 'P', 'Se'],
         ['Nb', 'Si', 'Se'],
         ['Nb', 'Ge', 'Se'],
         ['Cr', 'Si', 'Te', 'Mo'],
         ['Nb', 'Tc', 'Si', 'S'],
         ['Fe', 'Si', 'Ge', 'Te'],
         ['Ti', 'Si', 'Os', 'S'],
         ['Tc', 'Si', 'Ge', 'Mo', 'Se'],
         ['Mn', 'Tc', 'Si', 'Te'],
         ['Nb', 'Ge', 'S'],
         ['Cr', 'Si', 'Te', 'P'],
         ['Ti', 'Co', 'Si', 'S'],
         ['Cr', 'Si', 'Te', 'W'],
         ['Ge', 'Te', 'Mo'],
         ['Ge', 'Mo', 'Se'],
         ['Si', 'Mo', 'Se'],
         ['Cr', 'Ge', 'Te', 'P'],
         ['Cr', 'Ge', 'Se'],
         ['Cr', 'Si', 'Se'],
         ['Ge', 'Mo', 'S'],
         ['Si', 'Mo', 'S'],
         ['Cr', 'Ge', 'S'],
         ['Cr', 'Si', 'S'],
         ['Si', 'Te', 'Mo']]

mats=[]

for els in mat_els:
    As=[el for el in els if el in site_map["A"]]
    Bs=[el for el in els if el in site_map["B"]]
    Xs=[el for el in els if el in site_map["X"]]
    if len(As)==1:
        As.append(As[0])
    if len(Bs)==1:
        Bs.append(Bs[0])
    full_els=As+Bs+Xs
    mats.append(full_els)

for mat in mats:
    print(f" - {mat}")

In [5]:
ptable=pd.read_csv("periodictable.csv")
ptable["Mendeleev Element"]=ptable.apply(lambda x: element(x["Element"]), axis=1)
ptable["block"]=ptable.apply(lambda x: x["Mendeleev Element"].block, axis=1)
ptable["hardness"]=ptable.apply(lambda x: x["Mendeleev Element"].hardness(), axis=1)
ptable["econf"]=ptable.apply(lambda x: x["Mendeleev Element"].ec, axis=1)
ptable["nup"]=ptable.apply(lambda x: x["econf"].unpaired_electrons(), axis=1)
ptable["md_nvalence"]=ptable.apply(lambda x: x["Mendeleev Element"].nvalence(), axis=1)
ptable=ptable.set_index("Element")

In [6]:
key_values={
    "atomic_rad":ptable.atomic_radius,
    "atomic_vol":ptable.atomic_volume,
    "covalent_radius":ptable.covalent_radius_pyykko,
    "dipole":ptable.dipole_polarizability,
    "eaffinity":ptable.electron_affinity,
    "num_electrons":ptable.atomic_number,
    "vdwradius":ptable.vdw_radius,
    "e_negativity":ptable.en_allen,#Which one did Rhone use?
    "nvalence":ptable["md_nvalence"],#Which number used?
    "ion":ptable.first_ion_en,
    "hardness":ptable.hardness,
    "Nup":ptable.nup,
    "heat_of_formation":ptable["heat_of_formation"]
}

In [8]:
def weighted_average(x,elements,fraction):
    return sum([fraction[e]*key_values[x][e] for e in elements])

def weighted_std(x,elements,fraction,natoms=10):
    x_list=[]
    for e in elements:
        x_list+=[key_values[x][e]]*int(fraction[e]*natoms)
    return std(x_list)

def get_list(x,elements):
    return [key_values[x][e] for e in elements]

def raw_mean(x,elements):
    return mean(get_list(x,elements))

def raw_var(x,elements):
    x_l=get_list(x,elements)
    return var(x_l)/mean(x_l)**2

# def get_diffs(x,elements):
#     #takes in elements as [A1,A2,B1,B2,X]
#     x_list=get_list(x,elements)
#     for i in range(len(elements)):
#         for j in range(i+1,len(elements)):
            
#     diff_list=[abs(x_list[i]-x_list[j]) for i,j in diff_pairs]
#     return diff_list

# def sum_diff(x,elements):
#     return sum(get_diffs(x,elements))

# def var_diff(x,elements):
#     diffs=get_diffs(x,elements)
#     return var(diffs)/mean(diffs)**2

# def max_diff(x,elements):
#     return max(get_diffs(x,elements))

def get_stats(x,elements,fraction):
    """
    calculate the weighed mean, variance, and sum, variance, and maximum of differences
    of property x for elements elements
    
    return - (sum_diff,var_diff,max_diff,prop_variance)
    """
    weighted_avg=weighted_average(x,elements,fraction)
    
    x_list=get_list(x,elements)
    
    prop_var=var(x_list)/mean(x_list)**2
    
    #get diffs
    diff_list=[]
    for i in range(len(elements)):
        for j in range(i+1,len(elements)):
            diff_list.append(abs(x_list[i]-x_list[j]))
    
    sum_diff=sum(diff_list)
    var_diff=var(diff_list)/mean(diff_list)**2
    max_diff=max(diff_list)
    
    return weighted_avg,prop_var,sum_diff,var_diff,max_diff,x_list

def get_chem_info(elements,fractions):
    props=["atomic_rad","atomic_vol","covalent_radius","dipole","eaffinity","num_electrons","vdwradius","e_negativity","nvalence"]
    data={"elements": elements}
    for p in props:
        stats=get_stats(p,elements,fractions) #(weighted_avg, prop_var, sum_diff, var_diff, max_diff)
        data[p]=stats[5]
        data[f"{p}_sum_dif"]=stats[2]
        data[f"{p}_var_dif"]=stats[3]
        data[f"{p}_var"]=stats[1]
        data[f"{p}_avg"]=stats[0]
        data[f"{p}_max_dif"]=stats[4]
    
    ion_list=get_list("ion",elements)
    data["mean_ion"]=mean(ion_list)
    data["std_ion"]=std(ion_list)
    data["sum_ion"]=sum(ion_list)
    
    hardness_list=get_list("hardness",elements)
    data["hardness_mean"]=mean(hardness_list)
    data["hardness_var"]=var(hardness_list)

    Nup_list=get_list("Nup",elements)
    data["Nup_mean"]=mean(Nup_list)
    data["Nup_var"]=var(Nup_list)
    
    hof_list=get_list("heat_of_formation",elements)
    atomE_AB=hof_list[0]+hof_list[1]
    Born=atomE_AB+std(ion_list)-std(data["eaffinity"])
    data["Born"]=Born
    
    last_subshells=get_subshells(elements)
    num_p=0
    num_d=0
    num_f=0
    for shell,n in last_subshells:
        if shell[1]=='p':
            num_p+=n
        elif shell[1]=='d':
            num_d+=n
        elif shell[1]=='f':
            num_f+=n
    
    data["lastsubshell"]=last_subshells
    data["num_p"]=num_p
    data["num_d"]=num_d
    data["num_f"]=num_f
    
    return data

XSITES=["O","S","Se","Te","Po"]

def get_subshells(elements):
    
    last_subshells=[]
    for e in elements:
        ec=ptable.econf[e]
        if e not in XSITES:
            ec=ec.ionize(3)
        last_subshells.append(ec.last_subshell())
    return last_subshells

In [9]:
import re

def get_elements(formula):
    return re.split(r"\d+",formula)[:-1]



def get_fractions(formula):
    numbers=[float(n) for n in re.split(r"[a-zA-Z]+",formula)[1:]]
    fracs=array(numbers)/sum(numbers)
    elements=re.split(r"\d+",formula)[:-1]
    return {e: x for e,x in zip(elements,fracs)}

def construct_chemical_data(materials):
    dt_list=[]
    i=0
    for mat in materials:
        i+=1
        elems=get_elements(mat)
        fracs=get_fractions(mat)
        dt=get_chem_info(elems,fracs)
        dt["formula"]=mat
        dt["elem_frac"]=fracs
        dt_list.append(dt)
        if i%100==0:
            print(i)
    return dt_list

In [10]:
mats_org=['Cr1Co1Ge1Te6P1',
 'Cr1Co1Ge2Te6',
 'Cr1Co1Si1Ge1Te6',
 'Cr1Co1Si1Te6P1',
 'Cr1Co1Si2Te6',
 'Cr1Co1Te6P2',
 'Cr1Cu1Ge1Te6P1',
 'Cr1Cu1Ge2Te6',
 'Cr1Cu1Si1Ge1Te6',
 'Cr1Cu1Si1Te6P1',
 'Cr1Cu1Si2Te6',
 'Cr1Cu1Te6P2',
 'Cr1Fe1Ge1Te6P1',
 'Cr1Fe1Ge2Te6',
 'Cr1Fe1Si1Ge1Te6',
 'Cr1Fe1Si1Te6P1',
 'Cr1Fe1Si2Te6',
 'Cr1Fe1Te6P2',
 'Cr1Ge1Te6P1Ru1',
 'Cr1Ge2Te6Ru1',
 'Cr1Ni1Ge1Te6P1',
 'Cr1Ni1Ge2Te6',
 'Cr1Ni1Te6P2',
 'Cr1Si1Ge1Te6Ru1',
 'Cr1Si1Ni1Ge1Te6',
 'Cr1Si1Ni1Te6P1',
 'Cr1Si1Te6P1Ru1',
 'Cr1Si2Ni1Te6',
 'Cr1Si2Te6Ru1',
 'Cr1Te6P2Ru1',
 'Cr2Ge1Te6P1',
 'Cr2Ge2Te6',
 'Cr2Si1Ge1Te6',
 'Cr2Si1Te6P1',
 'Cr2Si2Te6',
 'Cr2Te6P2',
 'Nb1Cr1Ge1Te6P1',
 'Nb1Cr1Ge2Te6',
 'Nb1Cr1Si1Ge1Te6',
 'Nb1Cr1Si1Te6P1',
 'Nb1Cr1Si2Te6',
 'Nb1Cr1Te6P2',
 'Ti1Cr1Ge1Te6P1',
 'Ti1Cr1Ge2Te6',
 'Ti1Cr1Si1Ge1Te6',
 'Ti1Cr1Si1Te6P1',
 'Ti1Cr1Si2Te6',
 'Ti1Cr1Te6P2',
 'V1Cr1Ge1Te6P1',
 'V1Cr1Ge2Te6',
 'V1Cr1Si1Ge1Te6',
 'V1Cr1Si1Te6P1',
 'V1Cr1Si2Te6',
 'V1Cr1Te6P2',
 'Y1Cr1Ge1Te6P1',
 'Y1Cr1Ge2Te6',
 'Y1Cr1Si1Ge1Te6',
 'Y1Cr1Si1Te6P1',
 'Y1Cr1Si2Te6',
 'Y1Cr1Te6P2',
 'Ta1Cr1In1Te6Pb1',
 'Cr1In1Te6Mo1As1',
 'Cr1Tc1Sn2Te6',
 'Cr1Ge1Te6Mo1Pb1',
 'Cr1Ge1Te6Mo1As1',
 'Cr1Si1Sn1Te6Mo1',
 'Cr1In1Te6Mo1Pb1',
 'Ta1Cr1In1Si1Te6',
 'Sc1Cr1Sb1Te6Pb1',
 'Zr1Cr1Ge1Te6Pb1',
 'Zr1Cr1Si1Sb1Te6',
 'Hf1Cr1Si2Te6',
 'Hf1Al1Cr1Sb1Te6',
 'Zr1Cr1Si1Te6Pb1',
 'Sc1Cr1Sb1Te6P1',
 'Zr1Cr1Ge1Te6As1',
 'Cr1Ag1Ge1Te6P1',
 'Cr1Ag1Ge2Te6',
 'Cr1Ag1Sn1Ge1Te6',
 'Cr1Ag1Sn1Te6P1',
 'Cr1Ag1Sn2Te6',
 'Cr1Ag1Te6P2',
 'Cr1Ge1Te6Mo1P1',
 'Cr1Ge1Te6P1Au1',
 'Cr1Ge1Te6P1Ir1',
 'Cr1Ge1Te6P1Os1',
 'Cr1Ge1Te6P1Pd1',
 'Cr1Ge1Te6P1Pt1',
 'Cr1Ge1Te6P1Rh1',
 'Cr1Ge1Te6P1W1',
 'Cr1Ge2Te6Au1',
 'Cr1Ge2Te6Ir1',
 'Cr1Ge2Te6Mo1',
 'Cr1Ge2Te6Os1',
 'Cr1Ge2Te6Pd1',
 'Cr1Ge2Te6Pt1',
 'Cr1Ge2Te6Rh1',
 'Cr1Ge2Te6W1',
 'Cr1Re1Ge1Te6P1',
 'Cr1Re1Ge2Te6',
 'Cr1Re1Si1Ge1Te6',
 'Cr1Re1Si1Sn1Te6',
 'Cr1Re1Si1Te6P1',
 'Cr1Re1Si2Te6',
 'Cr1Re1Sn1Ge1Te6',
 'Cr1Re1Sn1Te6P1',
 'Cr1Re1Sn2Te6',
 'Cr1Re1Te6P2',
 'Cr1Si1Ag1Ge1Te6',
 'Cr1Si1Ag1Sn1Te6',
 'Cr1Si1Ag1Te6P1',
 'Cr1Si1Ge1Te6Au1',
 'Cr1Si1Ge1Te6Ir1',
 'Cr1Si1Ge1Te6Mo1',
 'Cr1Si1Ge1Te6Os1',
 'Cr1Si1Ge1Te6Pd1',
 'Cr1Si1Ge1Te6Pt1',
 'Cr1Si1Ge1Te6Rh1',
 'Cr1Si1Ge1Te6W1',
 'Cr1Si1Sn1Te6Au1',
 'Cr1Si1Sn1Te6Ir1',
 'Cr1Si1Sn1Te6Mo1',
 'Cr1Si1Sn1Te6Os1',
 'Cr1Si1Sn1Te6Pd1',
 'Cr1Si1Sn1Te6Pt1',
 'Cr1Si1Sn1Te6Rh1',
 'Cr1Si1Sn1Te6W1',
 'Cr1Si1Te6Mo1P1',
 'Cr1Si1Te6P1Au1',
 'Cr1Si1Te6P1Ir1',
 'Cr1Si1Te6P1Os1',
 'Cr1Si1Te6P1Pd1',
 'Cr1Si1Te6P1Pt1',
 'Cr1Si1Te6P1Rh1',
 'Cr1Si1Te6P1W1',
 'Cr1Si2Ag1Te6',
 'Cr1Si2Te6Au1',
 'Cr1Si2Te6Ir1',
 'Cr1Si2Te6Mo1',
 'Cr1Si2Te6Os1',
 'Cr1Si2Te6Pd1',
 'Cr1Si2Te6Pt1',
 'Cr1Si2Te6Rh1',
 'Cr1Si2Te6W1',
 'Cr1Sn1Ge1Te6Au1',
 'Cr1Sn1Ge1Te6Ir1',
 'Cr1Sn1Ge1Te6Mo1',
 'Cr1Sn1Ge1Te6Os1',
 'Cr1Sn1Ge1Te6Pd1',
 'Cr1Sn1Ge1Te6Pt1',
 'Cr1Sn1Ge1Te6Rh1',
 'Cr1Sn1Ge1Te6W1',
 'Cr1Sn1Te6Mo1P1',
 'Cr1Sn1Te6P1Au1',
 'Cr1Sn1Te6P1Ir1',
 'Cr1Sn1Te6P1Os1',
 'Cr1Sn1Te6P1Pd1',
 'Cr1Sn1Te6P1Pt1',
 'Cr1Sn1Te6P1Rh1',
 'Cr1Sn1Te6P1W1',
 'Cr1Sn2Te6Au1',
 'Cr1Sn2Te6Ir1',
 'Cr1Sn2Te6Mo1',
 'Cr1Sn2Te6Os1',
 'Cr1Sn2Te6Pd1',
 'Cr1Sn2Te6Pt1',
 'Cr1Sn2Te6Rh1',
 'Cr1Sn2Te6W1',
 'Cr1Tc1Ge1Te6P1',
 'Cr1Tc1Ge2Te6',
 'Cr1Tc1Si1Ge1Te6',
 'Cr1Tc1Si1Sn1Te6',
 'Cr1Tc1Si1Te6P1',
 'Cr1Tc1Si2Te6',
 'Cr1Tc1Sn1Ge1Te6',
 'Cr1Tc1Sn1Te6P1',
 'Cr1Tc1Sn2Te6',
 'Cr1Tc1Te6P2',
 'Cr1Te6Mo1P2',
 'Cr1Te6P2Au1',
 'Cr1Te6P2Ir1',
 'Cr1Te6P2Os1',
 'Cr1Te6P2Pd1',
 'Cr1Te6P2Pt1',
 'Cr1Te6P2Rh1',
 'Cr1Te6P2W1',
 'Hf1Cr1Ge1Te6P1',
 'Hf1Cr1Ge2Te6',
 'Hf1Cr1Si1Ge1Te6',
 'Hf1Cr1Si1Sn1Te6',
 'Hf1Cr1Si1Te6P1',
 'Hf1Cr1Si2Te6',
 'Hf1Cr1Sn1Ge1Te6',
 'Hf1Cr1Sn1Te6P1',
 'Hf1Cr1Sn2Te6',
 'Hf1Cr1Te6P2',
 'Sc1Cr1Ge1Te6P1',
 'Sc1Cr1Ge2Te6',
 'Sc1Cr1Si1Ge1Te6',
 'Sc1Cr1Si1Sn1Te6',
 'Sc1Cr1Si1Te6P1',
 'Sc1Cr1Si2Te6',
 'Sc1Cr1Sn1Ge1Te6',
 'Sc1Cr1Sn1Te6P1',
 'Sc1Cr1Sn2Te6',
 'Sc1Cr1Te6P2',
 'Ta1Cr1Ge1Te6P1',
 'Ta1Cr1Ge2Te6',
 'Ta1Cr1Si1Ge1Te6',
 'Ta1Cr1Si1Sn1Te6',
 'Ta1Cr1Si1Te6P1',
 'Ta1Cr1Si2Te6',
 'Ta1Cr1Sn1Ge1Te6',
 'Ta1Cr1Sn1Te6P1',
 'Ta1Cr1Sn2Te6',
 'Ta1Cr1Te6P2',
 'Zr1Cr1Ge1Te6P1',
 'Zr1Cr1Ge2Te6',
 'Zr1Cr1Si1Ge1Te6',
 'Zr1Cr1Si1Sn1Te6',
 'Zr1Cr1Si1Te6P1',
 'Zr1Cr1Si2Te6',
 'Zr1Cr1Sn1Ge1Te6',
 'Zr1Cr1Sn1Te6P1',
 'Zr1Cr1Sn2Te6',
 'Zr1Cr1Te6P2']

chem_dt=construct_chemical_data(mats_org)
chem_df=pd.DataFrame(chem_dt)

cols=['elem_frac','elements','formula','num_p','num_d', 'num_f', 'atomic_rad', 'atomic_vol', 'covalent_radius',
      'dipole', 'eaffinity', 'num_electrons', 'atomic_rad_sum_dif', 'atomic_rad_var_dif', 'atomic_rad_var',
      'atomic_rad_avg', 'atomic_rad_max_dif', 'atomic_vol_sum_dif', 'atomic_vol_var_dif', 'atomic_vol_var',
      'atomic_vol_avg', 'atomic_vol_max_dif', 'covalent_radius_sum_dif', 'covalent_radius_var_dif',
      'covalent_radius_var', 'covalent_radius_avg', 'covalent_radius_max_dif', 'dipole_sum_dif',
      'dipole_var_dif', 'dipole_var', 'dipole_avg', 'dipole_max_dif', 'eaffinity_sum_dif', 'eaffinity_var_dif',
      'eaffinity_var', 'eaffinity_avg', 'eaffinity_max_dif', 'num_electrons_sum_dif', 'num_electrons_var_dif',
      'num_electrons_var', 'num_electrons_avg', 'num_electrons_max_dif', 'vdwradius_sum_dif', 'vdwradius_var_dif',
      'vdwradius_var', 'vdwradius_avg', 'vdwradius_max_dif', 'e_negativity_sum_dif', 'e_negativity_var_dif',
      'e_negativity_var', 'e_negativity_avg', 'e_negativity_max_dif', 'nvalence_sum_dif', 'nvalence_var_dif',
      'nvalence_var', 'nvalence_avg', 'nvalence_max_dif', 'lastsubshell', 'std_ion', 'sum_ion', 'mean_ion', 'Born',
      'hardness_mean', 'hardness_var', 'Nup_mean','Nup_var']

100
200


In [13]:
from pymatgen.core.composition import Composition
import itertools

site_map={'A1':['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
                 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                 'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb'],
           'A2':['Sc', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni', 'Cu', 'Zn',
                 'Y', 'Zr', 'Nb', 'Mo', 'Tc', 'Ru', 'Rh', 'Pd', 'Ag', 'Cd',
                 'Lu', 'Hf', 'Ta', 'W', 'Re', 'Os', 'Ir', 'Pt', 'Au', 'Hg',
                 'La','Ce','Pr','Nd','Pm','Sm','Eu','Gd','Tb','Dy','Ho','Er','Tm','Yb'],
           'B1':['Ge','Si','P'],
           'B2':['Ge','Si','P'],
           'X':['S','Se','Te']}

mats_new=[]

for A1,A2,B1,B2,X in itertools.product(*site_map.values()):
    formula=Composition(f"{A1}1{A2}1{B1}1{B2}1{X}6").formula.replace(" ",'')
    if formula not in mats_org and formula not in mats_new:
        mats_new.append(formula)

In [14]:
len(mats_new)

17958

In [15]:
chem_dt_new=construct_chemical_data(mats_new)
print("Finished getting chemical data")
print("Constructing dataframe")
chem_df_new=pd.DataFrame(chem_dt_new)
print("done")

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800

C:\Users\peter\AppData\Local\Temp\ipykernel_14140\3324570548.py:59: RuntimeWarning: invalid value encountered in double_scalars
  var_diff=var(diff_list)/mean(diff_list)**2



3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
Finished getting chemical data
Constructing dataframe
done


In [3]:
chem_df_all=pd.concat([chem_df,chem_df_new],axis=0)[cols]

NameError: name 'chem_df' is not defined